In [ ]:
%pip install pandas

In [ ]:
import pandas as pd
import os

# Open vehicles dataset
ce = pd.read_csv('data/processed/veicoli/categorie_euro_2014_2021.csv')

# Open emissions dataset
se_folder = 'data/processed/emissioni'

for filename in os.listdir(se_folder):
    if filename.endswith('.csv'):
        # Read CSV file
        se = pd.read_csv(os.path.join(se_folder, filename))